In [2]:
#r "nuget: Shioaji, 1.1.8"

Installed Packages Shioaji, 1.1.8

In [3]:
using System.IO;
using Sinopac.Shioaji;
using System.Text.Json;
Shioaji _api = new Shioaji();

string jsonString = File.ReadAllText(@"D:\DotnetReactShioaji\DotnetReactShioaji\Sinopac.json");
JsonElement root = JsonDocument.Parse(jsonString).RootElement;
_api.Login(root.GetProperty("API_Key").GetString(), root.GetProperty("Secret_Key").GetString());

// var res = _api.Snapshots(new List<IContract>() {_api.Contracts.Futures["TXF"]["TXFR1"]}) //若誤用[0]就導它無法在array裡搜，硬用linq就報錯

Connecting as shioaji@sinopac on 210.59.255.161:80...


In [4]:
// struct basicData
// {
//     public string code;
//     public string direction;
//     public double entry_price;
//     public double cover_price;
//     public double pnl;
// }

public dynamic anonListProfitLossSummary(string bgn, string end)
{
    var src = _api.ListProfitLossSummary(bgn, end, _api.FutureAccount).profitloss_summary[0];
    var productAnonymous = new
    {
        code1 = (src.code.Substring(3, 5)) + (src.code[8] < 'L' ? "Call" : "Put"),
        direction = src.direction,
        entry_price = src.entry_price,
        cover_price = src.cover_price,
        netPnl = src.pnl - src.fee - src.tax
    };
    return productAnonymous;
}

// (, )
List<object> _temp = new List<object>();
_temp.Add(anonListProfitLossSummary("2024-01-10", "2024-01-10"));
_temp.Add(anonListProfitLossSummary("2024-01-18", "2024-01-18"));
_temp.Add(anonListProfitLossSummary("2024-01-31", "2024-01-31"));
_temp

index value 0 { code1 = 17550Put, direction = Buy, entry_price = 49.0, cover_price = 17481.0, netPnl = 951.0 } code1 17550Put direction Buy entry_price 49.0 cover_price 17481.0 netPnl 951.0 1 { code1 = 17250Call, direction = Buy, entry_price = 99.0, cover_price = 125.0, netPnl = 1259.0 } code1 17250Call direction Buy entry_price 99.0 cover_price 125.0 netPnl 1259.0 2 { code1 = 17900Call, direction = Buy, entry_price = 0.0, cover_price = 19.0, netPnl = -966.0 } code1 17900Call direction Buy entry_price 0.0 cover_price 19.0 netPnl -966.0

In [ ]:
// System.Collections.Generic.  List<Sinopac.Shioaji.FutureProfitLossSummary>
// 匿名型別的變數有著direction, tax..屬性，而屬性被賦值的動作叫object initializer syntax
List<FutureProfitLossSummary> a = _api.ListProfitLossSummary("2024-01-18", "2024-01-31", _api.FutureAccount).profitloss_summary;
foreach(var i in a)  Console.WriteLine(i);

In [ ]:
List<Sinopac.Shioaji.FuturePosition> _src = _api.ListPositions(_api.FutureAccount);  //允許多筆故idx 0會報錯
var direction = _src.Where(x => x.code.Contains("TX5")).Select(x => x.direction).FirstOrDefault();
var entry_price = _src.Where(x => x.code.Contains("TX5")).Select(x => x.price).FirstOrDefault();
var last_price = _src.Where(x => x.code.Contains("TX5")).Select(x => x.last_price).FirstOrDefault();
var Pnl = _src.Where(x => x.code.Contains("TX5")).Select(x => x.pnl).FirstOrDefault();
Pnl

In [ ]:
// var _contract = _api.Contracts.Futures["MXF"]["MXF202402"];    
// var _futOptOrder = new FutOptOrder()
// {
//     action = "Buy",
//     price = 18000,
//     quantity = 1, price_type = "LMT", order_type = "ROD"//, octype = OCType.DayTrade
// };
// var _trade = _api.PlaceOrder(_contract, _futOptOrder);
// _trade.GetType()
foreach(var i in _trade.status)
{
    Console.WriteLine(i);
}

In [ ]:
// _trade.order
foreach(var i in _trade.order)
{
    Console.WriteLine(i[0].value);
}

In [8]:
using System.Threading;

public void MxfMock(double argEntryPrice, double argStp)
{
    List<double> _temp = new List<double>();
    while (true)
    {
        double spotClose = (double)_api.Snapshots(new List<IContract>() { _api.Contracts.Futures["TXF"]["TXFR1"] })[0].close;
        _temp.Add(spotClose);
        _temp.Reverse();
        string result = string.Join(", ", _temp.Take(10));
        Console.WriteLine($"現在{spotClose}點 \t 過去 10 ticks {result}\n高點{_temp.Max()} since start");
        if (spotClose >= argEntryPrice + argStp)
        {
            argEntryPrice += 2;
            Console.WriteLine($"已拉開故到{argEntryPrice}才會被掃出場");
            while (true)
            {
                spotClose = (double)_api.Snapshots(new List<IContract>() { _api.Contracts.Futures["TXF"]["TXFR1"] })[0].close;
                _temp.Add(spotClose);
                if (spotClose <= (_temp.Max() - argStp))
                {
                    string desc = spotClose >= argEntryPrice ? "停利" : "保本";
                    Console.WriteLine($"★{desc}出在{spotClose}。");
                    break;
                }
                else if (spotClose <= argEntryPrice)
                {
                    Console.WriteLine($"★保本出在{spotClose}。");
                    break;
                }
                else
                {
                    Console.WriteLine(spotClose);
                    Thread.Sleep(5_000);
                }
            }
            break;
        }
        else if (spotClose <= argEntryPrice - argStp)
        {
            Console.WriteLine($"★停損，出在{spotClose}。");
            break;
        }
        else Thread.Sleep(5_000);
    }
    Console.WriteLine("★★小台積期：" + _api.Snapshots(new List<IContract>() { _api.Contracts.Futures["QFF"]["QFFR1"] })[0].close);
}


DateTime DTN = DateTime.Now;
bool preMktTime = DTN < DTN.Date + new TimeSpan(5, 00, 0);
bool onMktTime = (DTN > DTN.Date + new TimeSpan(8, 45, 0) && DTN < DTN.Date + new TimeSpan(13, 45, 0));
DateTime end = preMktTime ? DTN.Date + new TimeSpan(5, 00, 0) : (onMktTime ? DTN.Date + new TimeSpan(13, 45, 0) : DateTime.Now.Date + new TimeSpan(29, 0, 0));
while (DateTime.Now < end )
{
    // 1.有部位就依成本價
    // List<Sinopac.Shioaji.FuturePosition> _src = _api.ListPositions(_api.FutureAccount);
    // var Pnl = _src.Where(x => x.code.Contains("MXF")).Select(x => x.pnl).FirstOrDefault();
    // var entry_price = _src.Where(x => x.code.Contains("MXF")).Select(x => x.price).FirstOrDefault();
    

    // if(Pnl >= entry_price)
    // {
    //     MxfMock( rtPositions.First().price, decimal 10.0)
    // }
    if(false)
    {
        //
    }


    // 2.無部位就主觀抓底點
    else
    {
        MxfMock(17975.0, 10.0);
        break;
    }    
}

現在17975點 	 過去 10 ticks 17975
高點17975 since start
現在17974點 	 過去 10 ticks 17974, 17975
高點17975 since start
現在17974點 	 過去 10 ticks 17974, 17975, 17974
高點17975 since start
現在17974點 	 過去 10 ticks 17974, 17974, 17975, 17974
高點17975 since start
現在17974點 	 過去 10 ticks 17974, 17974, 17975, 17974, 17974
高點17975 since start
現在17973點 	 過去 10 ticks 17973, 17974, 17974, 17975, 17974, 17974
高點17975 since start
現在17974點 	 過去 10 ticks 17974, 17974, 17974, 17975, 17974, 17974, 17973
高點17975 since start
現在17974點 	 過去 10 ticks 17974, 17973, 17974, 17974, 17975, 17974, 17974, 17974
高點17975 since start
現在17974點 	 過去 10 ticks 17974, 17974, 17974, 17974, 17975, 17974, 17974, 17973, 17974
高點17975 since start
現在17973點 	 過去 10 ticks 17973, 17974, 17973, 17974, 17974, 17975, 17974, 17974, 17974, 17974
高點17975 since start
現在17973點 	 過去 10 ticks 17973, 17974, 17974, 17974, 17974, 17975, 17974, 17974, 17973, 17974
高點17975 since start
現在17973點 	 過去 10 ticks 17973, 17973, 17974, 17973, 17974, 17974, 17975, 17974, 1797

Error: Microsoft.CSharp.RuntimeBinder.RuntimeBinderException: Cannot perform runtime binding on a null reference
   at CallSite.Target(Closure, CallSite, Object)
   at System.Dynamic.UpdateDelegates.UpdateAndExecute1[T0,TRet](CallSite site, T0 arg0)
   at CallSite.Target(Closure, CallSite, Object)
   at Sinopac.Shioaji.SolaceApi.Snapshots(List`1 contracts)
   at Sinopac.Shioaji.Shioaji.Snapshots(List`1 contracts)
   at Submission#7.MxfMock(Double argEntryPrice, Double argStp)
   at Submission#7.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)